Cointab data analyst test
Client X tied with multiple courier company Charges depend on Weight of the product and distance between warehouse and customer delivery address(Zones)
We need to verify the charges levied by the partners per order are correct as per delivery.

Company X Order Report *{x1}*
Columns: ExternOrderNo, SKU, Order Qty.

Company X - SKU Master (Gross weight of each product) *{X2}*
Columns: SKU Weight (g)

Company X - Pincode Zones *{x3}*
Columns: Warehouse Pincode Customer Pincode Zone

Courier Company – Invoice *{c1}*
Columns: Order ID AWB Code Charged Weight Warehouse Pincode Customer Pincode Zone Type of Shipment

Courier Company – Rates *{c2}*
Given the Rates based on different slab weights, zones, and Type of shipment

Expected_Result
Order ID - in Company order report

AWB number - in Courier Company Invoice

Total weight as per X (KG) = Calculated based on weights for differet orders and their Quantity ( df4['Total weight as per X (KG)'] = df4["Order Qty"]* 0.001* df4["Weight (g)"] )

Weight slab as per X (KG) = round off by celing at 0.5 (lambda x: (math.ceil(x*2)/2) )

Total weight as per Courier Company (KG) = in Courier Company Invoice

Weight slab charged by Courier Company (KG) = round off by celing at 0.5 (lambda x: (math.ceil(x*2)/2) )

Delivery Zone as per X = in Company X - Pincode Zones

Delivery Zone charged by Courier Company = in Courier Company – Invoice

Expected Charge as per X (Rs.) = in c2 we have {a/b/c/d/e} zone and we have diffrent slab for zone and charge

Calculted based on weather the type of shipment in forword charges or Foreord and rto charges
after finding that find in which zone {a/b/c/d/e} then accordigly which slab gonna lay in that see value if so use frw_fixed or else use frw_fixed for first 0.5kg and frw_additional for remaining kgs. then based on the zone,map the rate and calculte
Difference Between Expected Charges and Billed Charges (Rs.) =

calucated by finding the difference between calculated and billed
Summery
We must Find and Conclude that the total orders are correctly charged, over charged , or under charged

In [113]:
import math
import pandas as pd
import numpy as np

In [348]:
# loading the data
x1=pd.read_csv("Company X - Order Report.csv")
x2=pd.read_csv("Company X - Pincode Zones.csv")
x3=pd.read_csv("Company X - SKU Master.csv")
c1=pd.read_csv("Courier Company - Rates.csv")
c2=pd.read_csv("Courier Company - Invoice.csv")

# Company X - Order Report.csv

In [349]:
# rename the ExternOrderNo to Order ID
x1=x1.rename(columns={"ExternOrderNo":"Order ID"})

In [243]:
x1

,Order ID,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
...,...,...,...
388,2001806229,8904223818942,1.0
389,2001806229,8904223818850,1.0
390,2001806226,8904223818850,2.0
391,2001806210,8904223816214,1.0


In [244]:
# data type of columns
x1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Order ID   393 non-null    int64  
 1   SKU        393 non-null    object 
 2   Order Qty  393 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 9.3+ KB


# Company X - Pincode Zones.csv

In [245]:
x2

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


In [246]:
#Checking unique value in each columns
x2.nunique()

Warehouse Pincode      1
Customer Pincode     108
Zone                   3
dtype: int64

In [247]:
x2['Zone'].value_counts()

b    79
d    38
e     7
Name: Zone, dtype: int64

In [248]:
#checking missing value
x2.isnull().sum()

Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64

In [249]:
#change the name of column "Zone" into "Delivery Zone as per X"
x2=x2.rename(columns={"Zone":"Delivery Zone as per X"})

In [250]:
x2.head()

,Warehouse Pincode,Customer Pincode,Delivery Zone as per X
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d


In [251]:
#Since 121003 pincode contains only "Warehouse Pincode", we droped this column
x2=x2.drop(["Warehouse Pincode"],axis=1)

In [252]:
x2

,Customer Pincode,Delivery Zone as per X
0,507101,d
1,486886,d
2,532484,d
3,143001,b
4,515591,d
...,...,...
119,325207,b
120,303702,b
121,313301,b
122,173212,e


In [253]:
#Checking Number of unique value in each column
x2.nunique()

Customer Pincode          108
Delivery Zone as per X      3
dtype: int64

# Company X - SKU Master.csv

In [254]:
x3.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [255]:
#Checking null values
x3.isnull().sum()

SKU           0
Weight (g)    0
dtype: int64

In [256]:
#Checking unique values in each column
x3.nunique()

SKU           65
Weight (g)    31
dtype: int64

In [257]:
#checking unique value in SKU column
x3["SKU"].value_counts()

GIFTBOX202002    2
8904223815682    1
8904223819291    1
8904223819116    1
8904223819161    1
                ..
8904223818997    1
8904223818942    1
8904223819024    1
8904223819031    1
SACHETS001       1
Name: SKU, Length: 65, dtype: int64

In [258]:
#since row no 52 and 56 are same, we droped one of them
x3.drop_duplicates(subset="SKU",inplace=True)

In [259]:
x3

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120
...,...,...
61,8904223819505,210
62,8904223819499,210
63,8904223819512,210
64,8904223819543,300


In [260]:
x1

,Order ID,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
...,...,...,...
388,2001806229,8904223818942,1.0
389,2001806229,8904223818850,1.0
390,2001806226,8904223818850,2.0
391,2001806210,8904223816214,1.0


In [261]:
# mergeing x1 and x3
df=pd.merge(x1,x3, on= "SKU")

In [262]:
df

,Order ID,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001821995,8904223818706,1.0,127
2,2001819252,8904223818706,1.0,127
3,2001816996,8904223818706,1.0,127
4,2001814580,8904223818706,1.0,127
...,...,...,...,...
388,2001806616,8904223819123,1.0,250
389,2001806567,8904223815804,1.0,160
390,2001806567,8904223818577,1.0,150
391,2001806408,8904223819437,2.0,552


In [263]:
df.isnull().sum()

Order ID      0
SKU           0
Order Qty     0
Weight (g)    0
dtype: int64

In [264]:
#Checking Null Values
df.nunique()

Order ID      124
SKU            65
Order Qty       6
Weight (g)     31
dtype: int64

In [265]:
#Add a new column "Total weight as per X (G)" from "Weight (g)" and "Order Qty"
df["Total weight as per X (G)"]=df["Weight (g)"]*df["Order Qty"]
#changing column name
df=df.rename(columns={"Total weight as per X (G)" : "Total weight as per X (KG)"})

In [266]:
df.head()

,Order ID,SKU,Order Qty,Weight (g),Total weight as per X (KG)
0,2001827036,8904223818706,1.0,127,127.0
1,2001821995,8904223818706,1.0,127,127.0
2,2001819252,8904223818706,1.0,127,127.0
3,2001816996,8904223818706,1.0,127,127.0
4,2001814580,8904223818706,1.0,127,127.0


In [267]:
#unit change G to KG
df["Total weight as per X (KG)"]=np.round(df['Total weight as per X (KG)']/1000,3)

In [268]:
df

,Order ID,SKU,Order Qty,Weight (g),Total weight as per X (KG)
0,2001827036,8904223818706,1.0,127,0.127
1,2001821995,8904223818706,1.0,127,0.127
2,2001819252,8904223818706,1.0,127,0.127
3,2001816996,8904223818706,1.0,127,0.127
4,2001814580,8904223818706,1.0,127,0.127
...,...,...,...,...,...
388,2001806616,8904223819123,1.0,250,0.250
389,2001806567,8904223815804,1.0,160,0.160
390,2001806567,8904223818577,1.0,150,0.150
391,2001806408,8904223819437,2.0,552,1.104


In [269]:
# grouping in order id and doing sum of total weight
df=df.groupby(['Order ID']).sum()

C:\Users\sony\AppData\Local\Temp\ipykernel_11032\995714521.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df=df.groupby(['Order ID']).sum()


In [270]:
df

,Order Qty,Weight (g),Total weight as per X (KG)
Order ID,,,
2001806210,2.0,220,0.220
2001806226,2.0,240,0.480
2001806229,3.0,500,0.500
2001806232,8.0,377,1.302
2001806233,2.0,245,0.245
...,...,...,...
2001821995,2.0,477,0.477
2001822466,10.0,352,1.376
2001823564,6.0,336,0.672


In [271]:
#reset index "Order ID"
df.reset_index(level=0, inplace=True)
df

,Order ID,Order Qty,Weight (g),Total weight as per X (KG)
0,2001806210,2.0,220,0.220
1,2001806226,2.0,240,0.480
2,2001806229,3.0,500,0.500
3,2001806232,8.0,377,1.302
4,2001806233,2.0,245,0.245
...,...,...,...,...
119,2001821995,2.0,477,0.477
120,2001822466,10.0,352,1.376
121,2001823564,6.0,336,0.672
122,2001825261,13.0,611,1.557


In [350]:
# rounding of with ceil fuction for sbove value
df["Weight slab as per X (KG)"] = df['Total weight as per X (KG)'].apply(lambda x: (math.ceil(x*2)/2) )

In [273]:
df.head()

,Order ID,Order Qty,Weight (g),Total weight as per X (KG),Weight slab as per X (KG)
0,2001806210,2.0,220,0.220,0.5
1,2001806226,2.0,240,0.480,0.5
2,2001806229,3.0,500,0.500,0.5
3,2001806232,8.0,377,1.302,1.5
4,2001806233,2.0,245,0.245,0.5


# Courier Company - Invoice.csv 

In [274]:
c2.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


In [275]:
c2.nunique()

AWB Code                124
Order ID                124
Charged Weight           54
Warehouse Pincode         1
Customer Pincode        108
Zone                      3
Type of Shipment          2
Billing Amount (Rs.)     20
dtype: int64

In [276]:
#checking unique value in Type of Shipment column
c2["Type of Shipment"].value_counts()

Forward charges            109
Forward and RTO charges     15
Name: Type of Shipment, dtype: int64

In [277]:
# droping this 2 colum dont want
df.drop(["Order Qty"],axis=1,inplace=True)

In [278]:
df.drop(["Weight (g)"],axis=1,inplace=True)

In [279]:
df

,Order ID,Total weight as per X (KG),Weight slab as per X (KG)
0,2001806210,0.220,0.5
1,2001806226,0.480,0.5
2,2001806229,0.500,0.5
3,2001806232,1.302,1.5
4,2001806233,0.245,0.5
...,...,...,...
119,2001821995,0.477,0.5
120,2001822466,1.376,1.5
121,2001823564,0.672,1.0
122,2001825261,1.557,2.0


In [280]:
#marging df and c2 on "Order ID" column
new=pd.merge(df,c2, on= "Order ID")

In [281]:
new

,Order ID,Total weight as per X (KG),Weight slab as per X (KG),AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3
...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.477,0.5,1091121183730,0.50,121003,342008,d,Forward charges,45.4
120,2001822466,1.376,1.5,1091121305541,1.10,121003,342301,d,Forward charges,135.0
121,2001823564,0.672,1.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8
122,2001825261,1.557,2.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0


In [282]:
#Rename column
new=new.rename(columns={"Charged Weight":"Total weight as per Courier Company (KG)","Zone":"Delivery Zone charged by Courier Company","Billing Amount (Rs.)":"Charges Billed by Courier Company (Rs.)"})


In [283]:
new.head()

,Order ID,Total weight as per X (KG),Weight slab as per X (KG),AWB Code,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.)
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3


In [284]:
#creating weight slab as per curier company
new['Weight slab charged by Courier Company (KG)'] = new['Total weight as per Courier Company (KG)'].apply(lambda x: (math.ceil(x*2)/2) )

In [285]:
new

,Order ID,Total weight as per X (KG),Weight slab as per X (KG),AWB Code,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab charged by Courier Company (KG)
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,3.0
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,1.0
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,1.0
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.5
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.477,0.5,1091121183730,0.50,121003,342008,d,Forward charges,45.4,0.5
120,2001822466,1.376,1.5,1091121305541,1.10,121003,342301,d,Forward charges,135.0,1.5
121,2001823564,0.672,1.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8,1.0
122,2001825261,1.557,2.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0,2.0


# Company X - Pincode Zones

In [286]:
x2

,Customer Pincode,Delivery Zone as per X
0,507101,d
1,486886,d
2,532484,d
3,143001,b
4,515591,d
...,...,...
119,325207,b
120,303702,b
121,313301,b
122,173212,e


In [287]:
#Checking Duplicate valuesx
x2["Customer Pincode"].value_counts()

302017    5
313001    4
302002    2
302031    2
248001    2
         ..
723146    1
140604    1
831006    1
562110    1
313301    1
Name: Customer Pincode, Length: 108, dtype: int64

In [288]:
#Checking all duplicate values
x2[x2["Customer Pincode"].duplicated()]

,Customer Pincode,Delivery Zone as per X
16,140301,b
50,248001,b
72,302002,b
75,302017,b
77,313001,b
78,313001,b
82,313001,b
90,302017,b
101,335001,b
106,302031,b


In [289]:
x2[x2["Customer Pincode"]==140301]

,Customer Pincode,Delivery Zone as per X
7,140301,b
16,140301,b


In [290]:
x2[x2["Customer Pincode"]==302017]

,Customer Pincode,Delivery Zone as per X
74,302017,b
75,302017,b
90,302017,b
113,302017,b
117,302017,b


In [291]:
#drop duplicate data
x2.drop_duplicates(subset="Customer Pincode", inplace=True)

In [292]:
x2.head()

,Customer Pincode,Delivery Zone as per X
0,507101,d
1,486886,d
2,532484,d
3,143001,b
4,515591,d


In [293]:
#marging new and x2 columns on "Customer Pincode"
new=pd.merge(new,x2, on= "Customer Pincode")

In [294]:
new.head()

,Order ID,Total weight as per X (KG),Weight slab as per X (KG),AWB Code,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab charged by Courier Company (KG),Delivery Zone as per X
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,3.0,b
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,1.0,d
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,1.0,d
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.5,d
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0,b


In [295]:
#Checking null values
new.isnull().sum()

Order ID                                       0
Total weight as per X (KG)                     0
Weight slab as per X (KG)                      0
AWB Code                                       0
Total weight as per Courier Company (KG)       0
Warehouse Pincode                              0
Customer Pincode                               0
Delivery Zone charged by Courier Company       0
Type of Shipment                               0
Charges Billed by Courier Company (Rs.)        0
Weight slab charged by Courier Company (KG)    0
Delivery Zone as per X                         0
dtype: int64

In [296]:
new.shape

(124, 12)

In [297]:
#Checking unique value in each columns
new.nunique()

Order ID                                       124
Total weight as per X (KG)                      68
Weight slab as per X (KG)                        7
AWB Code                                       124
Total weight as per Courier Company (KG)        54
Warehouse Pincode                                1
Customer Pincode                               108
Delivery Zone charged by Courier Company         3
Type of Shipment                                 2
Charges Billed by Courier Company (Rs.)         20
Weight slab charged by Courier Company (KG)      7
Delivery Zone as per X                           3
dtype: int64

In [298]:
#Checking unique values in "Order ID" column
new["Order ID"].value_counts()

2001806210    1
2001809794    1
2001811229    1
2001811192    1
2001811153    1
             ..
2001807036    1
2001807012    1
2001807004    1
2001806968    1
2001827036    1
Name: Order ID, Length: 124, dtype: int64

# Courier Company - Rates

In [299]:
c1.head()

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,a,0.50,29.5,23.6,13.6,23.6
1,b,1.00,33.0,28.3,20.5,28.3
2,c,1.25,40.1,38.9,31.9,38.9
3,d,1.50,45.4,44.8,41.3,44.8
4,e,2.00,56.6,55.5,50.7,55.5


In [300]:
new.head()

,Order ID,Total weight as per X (KG),Weight slab as per X (KG),AWB Code,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab charged by Courier Company (KG),Delivery Zone as per X
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,3.0,b
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,1.0,d
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,1.0,d
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.5,d
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0,b


In [301]:
new['Type of Shipment'].value_counts()

Forward charges            109
Forward and RTO charges     15
Name: Type of Shipment, dtype: int64

# from this we know that we are only delavering product in B D E

In [302]:
new['Delivery Zone as per X'].value_counts()

b    79
d    38
e     7
Name: Delivery Zone as per X, dtype: int64

In [308]:
c2

,Order ID,Total weight as per X (KG),Weight slab as per X (KG),AWB Code,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab charged by Courier Company (KG),Delivery Zone as per X
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,3.0,b
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,1.0,d
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,1.0,d
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.5,d
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.477,0.5,1091121183730,0.50,121003,342008,d,Forward charges,45.4,0.5,b
120,2001822466,1.376,1.5,1091121305541,1.10,121003,342301,d,Forward charges,135.0,1.5,b
121,2001823564,0.672,1.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8,1.0,d
122,2001825261,1.557,2.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0,2.0,d


In [351]:
c1

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,a,0.50,29.5,23.6,13.6,23.6
1,b,1.00,33.0,28.3,20.5,28.3
2,c,1.25,40.1,38.9,31.9,38.9
3,d,1.50,45.4,44.8,41.3,44.8
4,e,2.00,56.6,55.5,50.7,55.5


# we only have to find chsrges for B D E and watch for c1.csv because we have diffrent slab values for diffrent zone

In [319]:
#Determine "Expected Charge as per X (Rs.)" 
x_cost=[] #Creat a empty list
for i in new.index:
    l=new["Type of Shipment"][i]
    if l == 'Forward charges':     
        j=new["Delivery Zone as per X"][i]
        if j== 'b':
            k=new["Weight slab as per X (KG)"][i]
            if k<= 1:   # slab is 1 kg from c2 Weight Slabs for b
                p=33   #Taken from c2( Forward Fixed Charge 33.0    Forward Additional Weight Slab Charge 28.3)
                x_cost.append(p)
            else:
                p=33+((k/1)-1)*28.3  #Taken from c2( Forward Fixed Charge 33.0    Forward Additional Weight Slab Charge 28.3)
                x_cost.append(p)
        elif j=='d':
            k=new["Weight slab as per X (KG)"][i]
            if k<= 1.5:   # slab is 1.5 kg from c2 Weight Slabs for d
                p=45.4    #Taken from c2( Forward Fixed Charge 45.4   Forward Additional Weight Slab Charge 44.8)
                x_cost.append(p)
            else:
                p=45.4+((k/1.5)-1)*44.8  ##Taken from c2( Forward Fixed Charge 45.4   Forward Additional Weight Slab Charge 44.8)
                x_cost.append(p)                
        elif j=='e':      
            k=new["Weight slab as per X (KG)"][i]
            if k<= 2:       # slab is 2 kg from c2 Weight Slabs for e
                p=k*56.6   #Taken from c2( Forward Fixed Charge 56.6   Forward Additional Weight Slab Charge 55.5)
                x_cost.append(p)
            else:
                p=56.6+((k/2)-1)*55.5  #Taken from c2( Forward Fixed Charge 56.6   Forward Additional Weight Slab Charge 55.5)
                x_cost.append(p)
                    
    else:
        
        j=new["Delivery Zone as per X"][i]
        if j== 'b':
            k=new["Weight slab as per X (KG)"][i]
            if k<= 1:  # slab is 1 kg from c2 Weight Slabs for b
                p=(33+20.5) #from c2  fwd fixed 33.0    fwd additional 28.3  rto fixed 20.5    rto additional 28.3
                x_cost.append(p)
            else:
                p=(33+20.5)+(((k/1)-1)*(28.3+28.3))
                x_cost.append(p)
        elif j=='d':
            k=new["Weight slab as per X (KG)"][i]
            if k<= 1.5:   # slab is 1.5 kg from c2 Weight Slabs for d
                p=(45.4+41.3)  #fwd fixed 45.4    fwd additional 44.8  rto fixed 41.3    rto additional 44.8
                x_cost.append(p)
            else:
                p=(45.4+41.3)+(((k/1.5)-1)*(44.8+44.8))
                x_cost.append(p)                
        elif j=='e':
            k=new["Weight slab as per X (KG)"][i]
            if k<= 2:           # slab is 2 kg from c2 Weight Slabs for e
                p=(56.6+50.7)  #fwd fixed 56.6    fwd additional 55.5  rto fixed 50.7    rto additional 55.5
                x_cost.append(p)
            else:
                p=(56.6+50.7)+(((k/2)-1)*(55.5+55.5))
                x_cost.append(p) 
print("Code Succesfully Run")     

Code Succesfully Run


In [320]:
x_cost

[33,
 45.4,
 45.4,
 45.4,
 33,
 45.4,
 45.4,
 33,
 33,
 33,
 33,
 33,
 45.4,
 75.26666666666667,
 45.4,
 33,
 47.25,
 45.4,
 33,
 45.4,
 33,
 33,
 33,
 45.4,
 45.4,
 45.4,
 45.4,
 45.4,
 33,
 33,
 33,
 33,
 33,
 33,
 45.4,
 45.4,
 45.4,
 47.25,
 33,
 45.4,
 45.4,
 33,
 45.4,
 33,
 33,
 60.33333333333333,
 47.25,
 33,
 33,
 33,
 33,
 45.4,
 45.4,
 47.25,
 45.4,
 33,
 86.69999999999999,
 33,
 45.4,
 45.4,
 33,
 33,
 107.30000000000001,
 33,
 33,
 33,
 45.4,
 28.3,
 47.25,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 53.5,
 61.5,
 60.33333333333333,
 105.13333333333333,
 86.69999999999999,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 86.69999999999999,
 33,
 33,
 33,
 33,
 56.6,
 56.6,
 53.5,
 53.5,
 90.0,
 33,
 33,
 75.75,
 33,
 33,
 53.5,
 86.69999999999999,
 33,
 81.8,
 86.69999999999999,
 107.30000000000001,
 28.3,
 33,
 33,
 86.69999999999999,
 33,
 33,
 33,
 33,
 33,
 47.25,
 86.69999999999999,
 116.56666666666665,
 113.2]

In [323]:
len(x_cost)

124

In [324]:
#Add a new column "Expected Charge as per X (Rs.)" by x_cost list
new["Expected Charge as per X (Rs.)"]=x_cost

In [325]:
new

,Order ID,Total weight as per X (KG),Weight slab as per X (KG),AWB Code,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Expected Charge as per X (Rs.)
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,3.0,b,33.000000
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,1.0,d,45.400000
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,1.0,d,45.400000
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.5,d,45.400000
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0,b,33.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.477,0.5,1091121183730,0.50,121003,342008,d,Forward charges,45.4,0.5,b,33.000000
120,2001822466,1.376,1.5,1091121305541,1.10,121003,342301,d,Forward charges,135.0,1.5,b,47.250000
121,2001823564,0.672,1.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8,1.0,d,86.700000
122,2001825261,1.557,2.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0,2.0,d,116.566667


In [326]:
#Add a new column "Expected Charge as per X (Rs.)" by x_cost list
new["Expected Charge as per X (Rs.)"]=x_cost

In [327]:
new

,Order ID,Total weight as per X (KG),Weight slab as per X (KG),AWB Code,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Expected Charge as per X (Rs.)
0,2001806210,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,3.0,b,33.000000
1,2001806226,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,1.0,d,45.400000
2,2001806229,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,1.0,d,45.400000
3,2001806232,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.5,d,45.400000
4,2001806233,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0,b,33.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.477,0.5,1091121183730,0.50,121003,342008,d,Forward charges,45.4,0.5,b,33.000000
120,2001822466,1.376,1.5,1091121305541,1.10,121003,342301,d,Forward charges,135.0,1.5,b,47.250000
121,2001823564,0.672,1.0,1091121666133,0.70,121003,492001,d,Forward and RTO charges,172.8,1.0,d,86.700000
122,2001825261,1.557,2.0,1091121981575,1.60,121003,517128,d,Forward and RTO charges,345.0,2.0,d,116.566667


In [330]:
#Add a new column "Difference Between Expected Charges and Billed Charges (Rs.)"
new["Difference Between Expected Charges and Billed Charges (Rs.)"]=np.round(new["Expected Charge as per X (Rs.)"]-new["Charges Billed by Courier Company (Rs.)"],0)

In [331]:
#Checking all columns name
new.columns

Index(['Order ID', 'Total weight as per X (KG)', 'Weight slab as per X (KG)',
       'AWB Code', 'Total weight as per Courier Company (KG)',
       'Warehouse Pincode', 'Customer Pincode',
       'Delivery Zone charged by Courier Company', 'Type of Shipment',
       'Charges Billed by Courier Company (Rs.)',
       'Weight slab charged by Courier Company (KG)', 'Delivery Zone as per X',
       'Expected Charge as per X (Rs.)',
       'Difference Between Expected Charges and Billed Charges (Rs.)'],
      dtype='object')

In [332]:
#Creating final data frame 
final=new.loc[:,['Order ID','AWB Code','Total weight as per X (KG)','Weight slab as per X (KG)','Total weight as per Courier Company (KG)','Weight slab charged by Courier Company (KG)','Delivery Zone as per X','Delivery Zone charged by Courier Company','Expected Charge as per X (Rs.)','Charges Billed by Courier Company (Rs.)','Difference Between Expected Charges and Billed Charges (Rs.)']]

In [333]:
final

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.0,b,b,33.000000,174.5,-142.0
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,d,d,45.400000,90.2,-45.0
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,d,d,45.400000,90.2,-45.0
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,45.400000,135.0,-90.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,b,b,33.000000,61.3,-28.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,0.5,b,d,33.000000,45.4,-12.0
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,b,d,47.250000,135.0,-88.0
121,2001823564,1091121666133,0.672,1.0,0.70,1.0,d,d,86.700000,172.8,-86.0
122,2001825261,1091121981575,1.557,2.0,1.60,2.0,d,d,116.566667,345.0,-228.0


# OUTPUT DATA - 1

In [334]:
final

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.0,b,b,33.000000,174.5,-142.0
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,d,d,45.400000,90.2,-45.0
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,d,d,45.400000,90.2,-45.0
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,45.400000,135.0,-90.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,b,b,33.000000,61.3,-28.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,0.5,b,d,33.000000,45.4,-12.0
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,b,d,47.250000,135.0,-88.0
121,2001823564,1091121666133,0.672,1.0,0.70,1.0,d,d,86.700000,172.8,-86.0
122,2001825261,1091121981575,1.557,2.0,1.60,2.0,d,d,116.566667,345.0,-228.0


In [335]:
correctly_charged_order=final[final["Difference Between Expected Charges and Billed Charges (Rs.)"]==0]
correctly_charged_order

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
24,2001806686,1091117229555,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
35,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
38,2001808118,1091117435134,0.343,0.5,0.50,0.5,b,b,33.0,33.0,0.0
40,2001807186,1091117323812,0.500,0.5,0.50,0.5,d,d,45.4,45.4,0.0
42,2001807290,1091117324206,0.500,0.5,0.50,0.5,d,d,45.4,45.4,0.0
51,2001807930,1091117327695,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
62,2001808295,1091117435661,0.245,0.5,0.20,0.5,e,e,107.3,107.3,0.0


In [337]:
f1=correctly_charged_order["Charges Billed by Courier Company (Rs.)"].sum(axis = 0, skipna = True)
print("Number of correctly charged order id :",correctly_charged_order.shape[0])
print("Total orders ,  where X has been correctly charged :",f1)

Number of correctly charged order id : 7
Total orders ,  where X has been correctly charged : 367.3


In [340]:
overcharged_order=final[final["Difference Between Expected Charges and Billed Charges (Rs.)"]<0]
overcharged_order

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.0,b,b,33.000000,174.5,-142.0
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,d,d,45.400000,90.2,-45.0
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,d,d,45.400000,90.2,-45.0
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,45.400000,135.0,-90.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,b,b,33.000000,61.3,-28.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,0.5,b,d,33.000000,45.4,-12.0
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,b,d,47.250000,135.0,-88.0
121,2001823564,1091121666133,0.672,1.0,0.70,1.0,d,d,86.700000,172.8,-86.0
122,2001825261,1091121981575,1.557,2.0,1.60,2.0,d,d,116.566667,345.0,-228.0


In [341]:
f2=overcharged_order["Difference Between Expected Charges and Billed Charges (Rs.)"].sum(axis = 0, skipna = True)
print("Number of overcharged order id :",overcharged_order.shape[0])
print("Total Orders where X has been overcharged :",f2)

Number of overcharged order id : 116
Total Orders where X has been overcharged : -8010.0


In [342]:
undercharged_order=final[final["Difference Between Expected Charges and Billed Charges (Rs.)"]>0]
undercharged_order

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
95,2001811475,1091121844806,0.689,1.0,0.5,0.5,e,b,56.6,33.0,24.0


In [343]:
f3=undercharged_order["Difference Between Expected Charges and Billed Charges (Rs.)"].sum(axis = 0, skipna = True)
print("Number of undercharged order id :",undercharged_order.shape[0])
print("Total Orders where X has been undercharged",f3)

Number of undercharged order id : 1
Total Orders where X has been undercharged 24.0


# Create A Dataframe For Output Data - 2

In [344]:
df_ex=pd.DataFrame({"Headings":["Total orders where X has been correctly charged","Total Orders where X has been overcharged","Total Orders where X has been undercharged"],"Count":[7,116,1],"Amount (Rs.)":[f1,f2,f3]})

In [345]:
df_ex

,Headings,Count,Amount (Rs.)
0,Total orders where X has been correctly charged,7,367.3
1,Total Orders where X has been overcharged,116,-8010.0
2,Total Orders where X has been undercharged,1,24.0


In [353]:
df_ex.to_csv("Output_Data_2.csv")

In [352]:
final.to_csv("Output_Data_1.csv")
